# **Link Dataset**

https://drive.google.com/drive/folders/1YxW7r-Xj2il-Njkzo8Ed7BRsUqmPKMeE?usp=sharing

# **Scraping**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import selenium
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException 
from selenium import webdriver

import datetime


## **Initiate**

In [ ]:
s=Service("C:\SAINS DATA\chromedriver.exe")
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_argument('disable_infobars')
# options.headless = True

driver = webdriver.Chrome(service=s, options=options)
url="https://www.youtube.com/results?search_query=python+101"
driver.get(url)
driver.maximize_window()

## **Scroll**

In [ ]:
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.CONTROL + Keys.HOME)
# channel_vid = driver.find_elements(By.CSS_SELECTOR, 'a[id = "video-title"]')

## **Get Video Links**

In [ ]:
parents = driver.find_elements(By.CSS_SELECTOR, 'ytd-video-renderer[class = "style-scope ytd-item-section-renderer"]')
yt_link = []
yt_views = []

for parent in parents:
    yt_link_elem = parent.find_element(By.CSS_SELECTOR, 'a[id = "thumbnail"]')
    href = yt_link_elem.get_attribute('href')
    if 'shorts' not in href:
        yt_link.append(href)
    
        yt_views_elem = parent.find_element(By.CSS_SELECTOR, 'span[class = "inline-metadata-item style-scope ytd-video-meta-block"]')
        view_text = yt_views_elem.text
        yt_views.append(view_text)
        
    else : 
        print('membuang shorts')

## **Add to Dataframe**

In [ ]:
df = pd.DataFrame({'Link': yt_link,
                'Views': yt_views
                  })
df.head(20)

In [ ]:
print(len(yt_link))
print(len(yt_views))

## **Save to CSV**

In [ ]:
df.to_csv('youtube_link.csv', index = False)

In [ ]:
df2 = pd.read_csv('youtube_link.csv')
df2.head()

## **Get Date and Detail Views**

In [ ]:
detail_views = []
uploaded = []
for i in yt_link:
    driver.get(i)
    time.sleep(1.5)
    
    expand = driver.find_element(By.ID, 'expand')
    expand.click()
    time.sleep(0.1)
    
    tgl = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]')
    uploaded.append(tgl.text)
    
    penonton = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]')
    detail_views.append(penonton.text)
    
    if len(detail_views)%10 == 0 and len(uploaded)%10 == 0:
        print(len(detail_views), 'data telah terscrapping dari total', len(df), 'data')
     
    

## **Add to Dataframe**

In [ ]:
df2['Upload Date'] = uploaded
df2['Detail Views'] = detail_views
df2.head()

## **Save to CSV**

In [ ]:
df2.to_csv('youtube_link_detail.csv', index = False)
print('saved!')

## **Get Thumbnail**

In [ ]:
from pythumb import Thumbnail

for i in yt_link:
    try:
        t = Thumbnail(i)
        t.fetch()
        t.save('C:\TSD\youtube_thumbnail')
    except FileExistsError:
        pass

# **Preprocessing**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
from tqdm import tqdm
import cv2
import shutil

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from yellowbrick.cluster import KElbowVisualizer

from sklearn.cluster import KMeans

import tensorflow as tf

In [ ]:
df = pd.read_csv('youtube_link_detail.csv')
df.head(10)

## **Remove Duplicate**

In [ ]:
df['Link'].duplicated().sum()

In [ ]:
df_fix = df.drop_duplicates(subset=['Link'], keep='first')
df_fix['Link'].duplicated().sum()

In [ ]:
df_fix.info()

## **Detail Views in Integer**

In [ ]:
num_view = []
for i in df_fix.loc[:, ('Detail Views')]:
    num_view.append(i.split(' ')[0])

df_fix.loc[:, ('Detail Views')] = num_view   

In [ ]:
df_fix

## **Remove Comma**

In [ ]:
df_fix = df_fix.stack().str.replace(',','').unstack()
df_fix

## **Drop Videos with 0 Views**

In [ ]:
df_fix.drop(df_fix.index[df_fix['Detail Views'] == 'No'], inplace = True)

## **Adjust Data Type**

In [ ]:
df_fix['Detail Views'] = df_fix['Detail Views'].apply(int)
df_fix.info()

## **Remove Day and Month**

In [ ]:
tahun = []
for tanggal in df_fix['Upload Date']:
    tanggal_split = tanggal.split(' ')[-1]
    tahun.append(tanggal_split)

df_fix['Upload Date'] = tahun
df_fix.head()

## **Adjust Data Type**

In [ ]:
df_fix['Upload Date'] = df_fix['Upload Date'].apply(int)
df_fix.info()

## **Filter Years**

In [ ]:
df_fix.drop(df_fix.index[df_fix['Upload Date'] <= 2019], inplace = True)
df_fix['Upload Date'].value_counts()

In [ ]:
df_fix = df_fix.reset_index(drop=True)
df_fix.head()

## **Save to CSV**

In [ ]:
df_fix.to_csv('data bersih_2019-2022.csv', index=False)

# **EDA**

In [ ]:
df_fix['Detail Views'].describe().apply(lambda x: format(x, '2f'))

In [ ]:
df_sort_asc = df_fix.sort_values(by = 'Detail Views')[:150].reset_index(drop=True)
df_sort_asc

In [ ]:
#100 - 24999
# len(df_fix.loc[(df_fix['Detail Views'] >= 100) & (df_fix['Detail Views'] < 25000), :].sort_values(by = 'Detail Views'))
df_fix.loc[(df_fix['Detail Views'] >= 100) & (df_fix['Detail Views'] < 25000), :].sort_values(by = 'Detail Views')

In [ ]:
#25000 - 249999
len(df_fix.loc[(df_fix['Detail Views'] >= 25000) & (df_fix['Detail Views'] < 250000), :].sort_values(by = 'Detail Views'))

In [ ]:
# >250000
len(df_fix.loc[(df_fix['Detail Views'] >= 250000), :].sort_values(by = 'Detail Views'))

In [ ]:
class_names = ['Rendah', 'Sedang', 'Tinggi']
batas_bin = [100, 24999, 249999, 19000000]

In [ ]:
df_fix.drop(df_fix.index[df_fix['Detail Views'] < 100], inplace = True)
len(df_fix)

## **Binning**

In [ ]:
df_fix['Detail Views'] = pd.cut(df_fix['Detail Views'], batas_bin, labels=class_names)
df_fix['Detail Views'].value_counts()

## **Get Youtube Video Identifier**

In [ ]:
df_yt_code = df_fix.loc[:, ('Link', 'Detail Views')]
yt_code = []
for i in df_yt_code['Link']:
    yt_code.append(i.split('=')[1])

df_yt_code['Link'] = yt_code
df_yt_code.head()

In [ ]:
data_train = df_yt_code.sample(frac = 0.80, random_state=10)
data_val = df_yt_code.drop(data_train.index)

print(len(data_train) + len(data_val))

## **Labelling Data**

In [ ]:
# path_tinggi_train = 'train/Tinggi/'
# path_sedang_train = 'train/Sedang/'
# path_rendah_train = 'train/Rendah/'

# for index, row in data_train.iterrows():
#     if row['Detail Views'] == 'Tinggi':
#         shutil.move('youtube_thumbnail/'+ row['Link'] + '.jpg', path_tinggi_train)
#     elif row['Detail Views'] == 'Sedang':
#         shutil.move('youtube_thumbnail/'+ row['Link'] + '.jpg', path_sedang_train)
#     elif row['Detail Views'] == 'Rendah':
#         shutil.move('youtube_thumbnail/'+ row['Link'] + '.jpg', path_rendah_train)
    

In [ ]:
# path_tinggi_val = 'val/Tinggi/'
# path_sedang_val = 'val/Sedang/'
# path_rendah_val = 'val/Rendah/'

# for index, row in data_val.iterrows():
#     if row['Detail Views'] == 'Tinggi':
#         shutil.move('youtube_thumbnail/'+ row['Link'] + '.jpg', path_tinggi_val)
#     elif row['Detail Views'] == 'Sedang':
#         shutil.move('youtube_thumbnail/'+ row['Link'] + '.jpg', path_sedang_val)
#     elif row['Detail Views'] == 'Rendah':
#         shutil.move('youtube_thumbnail/'+ row['Link'] + '.jpg', path_rendah_val)
    

In [ ]:
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
print(class_names_label)
nb_classes = len(class_names)

IMAGE_SIZE = (640, 360)

## **Load Data**

In [ ]:
def load_data():
    
    datasets = ['train', 'val']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'uint8')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
n_train = train_labels.shape[0]
n_test = test_labels.shape[0]

print ("Number of training examples: {}".format(n_train))
print ("Number of testing examples: {}".format(n_test))
print ("Each image is of size: {}".format(train_images.shape))

In [ ]:
train_images

In [ ]:
_, train_counts = np.unique(train_labels, return_counts=True)
_, test_counts = np.unique(test_labels, return_counts=True)
pd.DataFrame({'train': train_counts,
                'test': test_counts} 
                ).plot.bar()
plt.show()

In [ ]:
def display_examples(class_names, images, labels):
    """
        Display 25 images from the images array with its corresponding labels
    """
    
    fig = plt.figure(figsize=(10,10))
    fig.suptitle("Some examples of images of the dataset", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
display_examples(class_names, train_images, train_labels)

In [ ]:
def RGB2HEX(color):
    return "#{:02x}{:02x}{:02x}".format(int(color[0]), int(color[1]), int(color[2]))

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMAGE_SIZE) 
        if img is not None:
            images.append(img)
    return np.array(images, dtype='uint8')

In [ ]:
all_images = np.concatenate((train_images, test_images), axis=0)

rendah_images = np.concatenate((load_images_from_folder('train/Rendah/'), 
                                load_images_from_folder('val/Rendah/')), axis=0)

sedang_images = np.concatenate((load_images_from_folder('train/Sedang/'), 
                                load_images_from_folder('val/Sedang/')), axis=0)

tinggi_images = np.concatenate((load_images_from_folder('train/Tinggi/'), 
                                load_images_from_folder('val/Tinggi/')), axis=0)


In [ ]:
all_images_reshape = all_images.reshape(-1, all_images.shape[1]*all_images.shape[2]*all_images.shape[3])
rendah_reshape = rendah_images.reshape(-1, rendah_images.shape[1]*rendah_images.shape[2]*rendah_images.shape[3])
sedang_reshape = sedang_images.reshape(-1, sedang_images.shape[1]*sedang_images.shape[2]*sedang_images.shape[3])
tinggi_reshape = tinggi_images.reshape(-1, tinggi_images.shape[1]*tinggi_images.shape[2]*tinggi_images.shape[3])

## **Dominant Colors**

### **All Images**

In [ ]:
number_of_colors = 5
clf = KMeans(n_clusters = number_of_colors, random_state = 10)
labels = clf.fit_predict(all_images_reshape)
# labels_rendah = clf.fit_predict(rendah_reshape)
# labels_sedang = clf.fit_predict(sedang_reshape)
# labels_tinggi = clf.fit_predict(tinggi_reshape)

In [ ]:
from collections import Counter

counts = Counter(labels)

center_colors = clf.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors[i] for i in counts.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts.keys()]
print(hex_colors)
rgb_colors = [ordered_colors[i] for i in counts.keys()]

plt.figure(figsize = (8, 6))
plt.pie(counts.values(), labels = hex_colors, colors = hex_colors, autopct='%.0f%%')
plt.title('Most Used Colors of All Images')
plt.show()



### **Rendah**

In [ ]:
clf2 = KMeans(n_clusters = number_of_colors, random_state = 10)
labels_rendah = clf2.fit_predict(rendah_reshape)

In [ ]:
counts_rendah = Counter(labels_rendah)

center_colors2 = clf2.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors2[i] for i in counts_rendah.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts_rendah.keys()]
rgb_colors = [ordered_colors[i] for i in counts_rendah.keys()]

plt.figure(figsize = (8, 6))
plt.pie(counts_rendah.values(), labels = hex_colors, colors = hex_colors, autopct='%.0f%%')
plt.title('Most Used Colors of Images with Rendah Label')
plt.show()

### **Sedang**

In [ ]:
clf1 = KMeans(n_clusters = number_of_colors, random_state = 10)
labels_sedang = clf1.fit_predict(sedang_reshape)

In [ ]:
counts_sedang = Counter(labels_sedang)

center_colors1 = clf1.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors1[i] for i in counts_sedang.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts_sedang.keys()]
rgb_colors = [ordered_colors[i] for i in counts_sedang.keys()]

plt.figure(figsize = (8, 6))
plt.pie(counts_sedang.values(), labels = hex_colors, colors = hex_colors, autopct='%.0f%%')
plt.title('Most Used Colors of Images with Sedang Label')
plt.show()

### **Tinggi**

In [ ]:
clf3 = KMeans(n_clusters = number_of_colors, random_state = 10)
labels_tinggi = clf3.fit_predict(tinggi_reshape)

In [ ]:
counts_tinggi = Counter(labels_tinggi)

center_colors3 = clf3.cluster_centers_
# We get ordered colors by iterating through the keys
ordered_colors = [center_colors3[i] for i in counts_tinggi.keys()]
hex_colors = [RGB2HEX(ordered_colors[i]) for i in counts_tinggi.keys()]
rgb_colors = [ordered_colors[i] for i in counts_tinggi.keys()]

plt.figure(figsize = (8, 6))
plt.pie(counts_tinggi.values(), labels = hex_colors, colors = hex_colors, autopct='%.0f%%')
plt.title('Most Used Colors of Images with Tinggi Label')
plt.show()

# **Analisis Clustering**

## **Normalization**

In [ ]:
train_images = train_images/255
test_images = test_images/255

## **PCA**

In [ ]:
X = train_images.reshape(-1, train_images.shape[1]*train_images.shape[2]*train_images.shape[3])
print('ukuran data train (fitur):', X.shape)

X_scaling = StandardScaler().fit_transform(X)
    

In [ ]:
pca = PCA().fit(X_scaling)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

In [ ]:
pca = PCA(n_components=200, random_state=10)
X_pca = pca.fit_transform(X)
print('dimensi data setelah PCA:', X_pca.shape)

approximation = pca.inverse_transform(X_pca)
plt.figure(figsize=(8,4))
n = 200
plt.subplot(1, 2, 1)
plt.imshow(X[n].reshape(train_images.shape[1], train_images.shape[2], train_images.shape[3]), cmap=plt.get_cmap('gray'))
plt.xlabel(str(X.shape[1])+'components', fontsize = 14)
plt.title('original image', fontsize = 20)
plt.grid(False)
plt.subplot(1, 2, 2)
plt.imshow(approximation[n].reshape(train_images.shape[1], train_images.shape[2], train_images.shape[3]), cmap=plt.get_cmap('gray'))
plt.xlabel(str(X_pca.shape[1])+'components', fontsize = 14)
plt.title('image after pca', fontsize = 14)
plt.grid(False)
plt.show()

## **Determine Number of Cluster**

In [ ]:
visualizer = KElbowVisualizer(KMeans(random_state=10, max_iter = 1000), k=(1,11))

visualizer.fit(X_pca)
visualizer.show()

## **K-Means**

In [ ]:
kmeans = KMeans(n_clusters=3, n_jobs=-1, random_state=21, max_iter = 1000)
kmeans.fit(X_pca)

In [ ]:
y_kmeans = kmeans.fit_predict(X_pca)

## **K-Means Visualization**

In [ ]:
plt.figure(figsize=(9, 7))
plt.scatter(X_pca[y_kmeans==0, 0], X_pca[y_kmeans==0, 1], s=100, c='red', label ='Cluster 1')
plt.scatter(X_pca[y_kmeans==1, 0], X_pca[y_kmeans==1, 1], s=100, c='blue', label ='Cluster 2')
plt.scatter(X_pca[y_kmeans==2, 0], X_pca[y_kmeans==2, 1], s=100, c='green', label ='Cluster 3')
# plt.scatter(X_pca[y_kmeans==3, 0], X_pca[y_kmeans==3, 1], s=100, c='cyan', label ='Cluster 4')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='yellow', label = 'Centroids')
plt.legend(loc = 4)
plt.grid(None)
plt.show()

## **Cluster Analysis**

In [ ]:
# Jumlah data pada setiap cluster
clustering_label = pd.DataFrame(y_kmeans)
clustering_label.value_counts()

In [ ]:
# Melihat data hasil cluster
Z = kmeans.predict(X_pca)

for i in range(0,3):
    image_label = []
    row = np.where(Z==i)[0]
    num = row.shape[0]          
    r = int(np.floor(num/10.))    
    
    print("CLUSTER "+str(i))
    print(str(num)+" elements")

    plt.figure(figsize=(12,10))
    for k in range(0, num):
        plt.subplot(r+1, 10, k+1)
        image = train_images[row[k], ]
        image = image.reshape(360, 640, 3)
        image_label.append(train_labels[row[k], ])
        plt.imshow(image, cmap=plt.cm.binary)
        plt.axis('off')
    plt.show()
    print('Frequency of Each Category:')
    print(pd.Series(image_label).value_counts())
    print()

# **Analisis Klasifikasi**

## **Metode Machine Learning**

### **Fitur Ekstraksi Tekstur HOG**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.datasets import mnist
from tensorflow import keras
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, make_scorer
from sklearn.metrics import precision_recall_fscore_support as score
import itertools
from sklearn.metrics import classification_report
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import cv2
import shutil
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('youtube_link_detail.csv')
df.head()

**Fitur Ekstraksi Warna LAB-RGB**

In [ ]:
labels = ['Rendah','Sedang','Tinggi']

x_train = []
y_train = []

x_test = []
y_test = []

image_size = 400 

# All Data
for i in labels:
    folderPath = os.path.join(r"E:\00 KULYAH\SEMESTER 5\Data Mining II\UAS\Projek UAS\train",i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        try:
            img = cv2.resize(img,(image_size,image_size))
        except:
            print(i, j)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        x_train.append(img)
        y_train.append(i)
print('Data train DONE!')

# All Data
for i in labels:
    folderPath = os.path.join(r"E:\00 KULYAH\SEMESTER 5\Data Mining II\UAS\Projek UAS\val",i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        try:
            img = cv2.resize(img,(image_size,image_size))
        except:
            print(i, j)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        x_test.append(img)
        y_test.append(i)
print('Data train DONE!')

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

**Preprocessing**

In [ ]:
x_train.shape
# y_train.shape

In [ ]:
df['Link'].duplicated().sum()

In [ ]:
df.info()

In [ ]:
class_names = ['Rendah', 'Sedang', 'Tinggi']
batas_bin = [100, 24999, 249999, 19000000]

In [ ]:
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
print(class_names_label)
nb_classes = len(class_names)

**Normalisasi**

In [ ]:
images_train_norm = []
images_test_norm = []

for i in x_train : 
    norm_train = i/255.0
    images_train_norm.append(norm_train)

for i in x_test: 
    norm_test = i/255.0
    images_test_norm.append(norm_test)

images_train_norm[0]

**HOG**

In [ ]:
from skimage.feature import hog
x_train_hog = []
for i in range(len(x_train)):
    fd , hog_im = hog(x_train[i] , orientations=9 , pixels_per_cell = (8,8),
                     cells_per_block = (2,2) , visualize = True ,  multichannel = True)
    x_train_hog.append(fd)

x_train_hog = np.array(x_train_hog)  

In [ ]:
x_test_hog = []
for i in range(len(x_test)):
    fd , hog_im = hog(x_test[i] , orientations=9 , pixels_per_cell = (8,8),
                     cells_per_block = (2,2) , visualize = True ,  multichannel = True)
    x_test_hog.append(fd)

x_test_hog = np.array(x_test_hog)  

In [ ]:
import random
def random_image_to_hog():
    rnd = random.randint(0, 267)
    img = x_train[rnd]
    label = y_train
    classes = ['Rendah', 'Sedang', 'Tinggi']
    
    fd, hog_img = hog(img, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), visualize=True, multichannel = True)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(img, cmap=plt.cm.gray)
    ax1.set_title(classes)
        
    ax2.axis('off')
    ax2.imshow(hog_img, cmap=plt.cm.gray)
    ax2.set_title('HOG image')
    plt.show()

random_image_to_hog()

### **Model**

In [ ]:
# Hasil Perbandingan Performance Measure
model_performance = pd.DataFrame(columns=['Accuracy','Sensitivity', 'Specificity', 'Precision','F1-Score'])
# Hasil Perbandingan Accuracy Data Testing dan Data Training
model_performance_tr = pd.DataFrame(columns=['Accuracy Training', 'Accuracy Testing'])

In [ ]:
y_train = y_train.reshape(-1,1)
y_train

In [ ]:
y_train = y_train.astype('uint8')
y_train

In [ ]:
len(x_train_hog)

**SVM**

In [ ]:
# Model
from sklearn import svm
model = svm.SVC(kernel = 'rbf')
model.fit(x_train_hog , y_train.flatten())
y_pred_svm= model.predict(x_test_hog)
y_pred_tr= model.predict(x_train_hog)

# Performance Measure
print("\n => Classification Report for Support Vector Machine:\n%s"%classification_report(y_test.flatten(), y_pred_svm))
cm_svm=confusion_matrix(y_test.flatten(), y_pred_svm)
sns.heatmap(cm_svm,cmap='BuPu',annot=True,fmt='d')
print("\n => Confusion Matrix for Support Vector Machine:")

**Random Forest**

In [ ]:
# Model
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=0).fit(x_train_hog , y_train.flatten())
y_pred_rfc= rfc.predict(x_test_hog)
y_pred_tr= rfc.predict(x_train_hog)

# Performance Measure
print("\n => Classification Report for Random Forest:\n%s"%classification_report(y_test.flatten(), y_pred_rfc))
cm_svm=confusion_matrix(y_test.flatten(), y_pred_rfc)
sns.heatmap(cm_svm,cmap='BuPu',annot=True,fmt='d')
print("\n => Confusion Matrix for Random Forest:")

### **Fitur Ekstraksi Tekstur GLCM**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from keras.datasets import mnist
from tensorflow import keras
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, make_scorer
from sklearn.metrics import precision_recall_fscore_support as score
import itertools
from sklearn.metrics import classification_report
from numpy import mean
from numpy import std
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import tensorflow as tf
import skimage
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
from tqdm import tqdm
import cv2
import shutil

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

**Connect to GDrive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Fitur Ekstraksi Warna LAB-RGB**

In [ ]:
labels = ['Rendah','Sedang','Tinggi']

x_train = []
y_train = []

x_test = []
y_test = []

image_size = 400 

# All Data
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/PROJECT UAS/train/',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        try:
            img = cv2.resize(img,(image_size,image_size))
        except:
            print(i, j)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        x_train.append(img)
        y_train.append(i)
print('Data train DONE!')

# All Data
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/PROJECT UAS/val/',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        try:
            img = cv2.resize(img,(image_size,image_size))
        except:
            print(i, j)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        x_test.append(img)
        y_test.append(i)
print('Data train DONE!')

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

**Fitur Ekstraksi Warna LAB-RGB-HSV**

In [ ]:
labels = ['Rendah','Sedang','Tinggi']

x_train = []
y_train = []

x_test = []
y_test = []

image_size = 400 

# All Data
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/PROJECT UAS/train/',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        try:
            img = cv2.resize(img,(image_size,image_size))
        except:
            print(i, j)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        x_train.append(img)
        y_train.append(i)
print('Data train DONE!')

# All Data
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/PROJECT UAS/val/',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        try:
            img = cv2.resize(img,(image_size,image_size))
        except:
            print(i, j)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        x_test.append(img)
        y_test.append(i)
print('Data train DONE!')

x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

**GLCM**

In [ ]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    # menghitung matrix GLCM dan matrix texture
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all texture properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls_xtrain = []
for img, label in zip(img_train, y_train): 
    glcm_all_agls_xtrain.append(
            calc_glcm_all_agls(img, 
                               label, 
                               props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")


**Data Train**

In [ ]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_train = pd.DataFrame(glcm_all_agls_xtrain, 
                             columns = columns)

glcm_train.head()

In [ ]:
# Mengubah menjadi encoding
glcm_train.loc[glcm_train["label"] == "Rendah", "label"] = 0
glcm_train.loc[glcm_train["label"] == "Sedang", "label"] = 1
glcm_train.loc[glcm_train["label"] == "Tinggi", "label"] = 2

**Data Test**

In [ ]:
img_test = []

for img in x_test:
    # convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgs = gray/255
    imgs = imgs.astype(np.uint8)
    # h, w = gray.shape
    # ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
    # crop ROI
    # crop = gray[ymin:ymax, xmin:xmax]
    # resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)
    # save to list
    img_test.append(imgs)

In [ ]:
from skimage.feature import greycomatrix, greycoprops

# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    
    # menghitung matrix GLCM dan matrix texture
    glcm = greycomatrix(imgs, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label) 
    
    return feature


# ----------------- call calc_glcm_all_agls() for all texture properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls_xtest = []
for img, label in zip(img_test, y_test): 
    glcm_all_agls_xtest.append(
            calc_glcm_all_agls(img, 
                               label, 
                               props=properties)
                            )
 
columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)
        
columns.append("label")


In [ ]:
import pandas as pd 

# Create the pandas DataFrame for GLCM features data
glcm_test = pd.DataFrame(glcm_all_agls_xtest, 
                             columns = columns)

glcm_test

In [ ]:
# Mengubah menjadi encoding
glcm_test.loc[glcm_test["label"] == "Rendah", "label"] = 0
glcm_test.loc[glcm_test["label"] == "Sedang", "label"] = 1
glcm_test.loc[glcm_test["label"] == "Tinggi", "label"] = 2

**Mengubah Tipe Data**

In [ ]:
xg_train = glcm_train.drop(columns = ['label']).astype('float32').values
yg_train = glcm_train['label'].astype('float32').values
xg_test = glcm_test.drop(columns = ['label']).astype('float32').values
yg_test = glcm_test['label'].astype('float32').values

### **Model**

**SVM**

In [ ]:
from sklearn.svm import SVC 
svm_glcm = SVC(random_state = 77)
svm_glcm.fit(xg_train, yg_train)

pred_svm_glcm = svm_glcm.predict(xg_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_svm_glcm = round(accuracy_score(yg_test, pred_svm_glcm),2)
precision_svm_glcm = round(precision_score(yg_test, pred_svm_glcm, average='weighted'),2)
recall_svm_glcm = round(recall_score(yg_test, pred_svm_glcm, average='weighted'),2)
f1score_svm_glcm = round(f1_score(yg_test, pred_svm_glcm, average='weighted'),2)


**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_glcm = LogisticRegression(random_state = 77)
lr_glcm.fit(xg_train, yg_train)

pred_lr_glcm = lr_glcm.predict(xg_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_lr_glcm = round(accuracy_score(yg_test, pred_lr_glcm),2)
precision_lr_glcm = round(precision_score(yg_test, pred_lr_glcm, average='weighted'),2)
recall_lr_glcm = round(recall_score(yg_test, pred_lr_glcm, average='weighted'),2)
f1score_lr_glcm = round(f1_score(yg_test, pred_lr_glcm, average='weighted'),2)

**XGBoost**

In [ ]:
import xgboost as xgb
xgb_glcm = xgb.XGBClassifier(random_state = 77)
xgb_glcm.fit(xg_train, yg_train)

pred_xgb_glcm = xgb_glcm.predict(xg_test)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_xgb_glcm = round(accuracy_score(yg_test, pred_xgb_glcm),2)
precision_xgb_glcm = round(precision_score(yg_test, pred_xgb_glcm, average='weighted'),2)
recall_xgb_glcm = round(recall_score(yg_test, pred_xgb_glcm, average='weighted'),2)
f1score_xgb_glcm = round(f1_score(yg_test, pred_xgb_glcm, average='weighted'),2)

**Komparasi Model**

In [ ]:
compare_model1 = pd.DataFrame({'Model':['GLCM - XGBoost', 'GLCM - Logistic Regression', 'GLCM - Support Vector Machine'],
                           'Accuracy':[akurasi_xgb_glcm, akurasi_lr_glcm, akurasi_svm_glcm],
                           'Recall':[recall_xgb_glcm, recall_lr_glcm, recall_svm_glcm],
                           'Precision':[precision_xgb_glcm, precision_lr_glcm, precision_svm_glcm],
                           'F1-Score':[f1score_xgb_glcm, f1score_lr_glcm, f1score_svm_glcm]})

compare_model1 = compare_model1.sort_values(by=['Accuracy', 'Precision', 'Recall', 'F1-Score'],ascending = [False, False, False, False])
compare_model1

## **Metode Deep Learning**

### **Klasik CNN**

### **Arsitektur 1**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (360, 640, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation=tf.nn.relu),
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, batch_size=16, epochs=20, validation_split = 0.2)

In [ ]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['acc'],'bo--', label = "acc")
    plt.plot(history.history['val_acc'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history)

In [ ]:
test_loss = model.evaluate(test_images, test_labels)

### **Arsitektur 2**

In [ ]:
import keras
import time
from tensorflow import keras
from keras import Sequential
from keras import layers
import tensorflow as tf
# import tensorflow_addons as tfa
# from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout,SeparableConv2D, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG


In [ ]:
def create_model():
    shape_img = (360,640,3)
    
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=shape_img, activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=shape_img, activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=shape_img, activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=shape_img, activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=shape_img, activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=shape_img, activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [ ]:
model = create_model()
hists = []

# divisor = 5

start_time = time.time()
# X_train, y_train = load_img(cut_df(df,divisor,1))
# y_train = to_categorical(y_train)

callbacks = [EarlyStopping(monitor='val_loss', patience=20),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

model.fit(train_images, train_labels, batch_size=128, epochs=10, callbacks=callbacks, validation_split = 0.1, verbose = 1)
hists.append(model.history.history)


In [ ]:
acc = []
val_acc = []
for i in range(len(hists)):
    acc += hists[i]["acc"]
    val_acc += hists[i]["val_acc"]
hist_df = pd.DataFrame({"# Epoch": [e for e in range(1,len(acc)+1)],"Accuracy": acc, "Val_accuracy": val_acc})
hist_df.plot(x = "# Epoch", y = ["Accuracy","Val_accuracy"])
plt.title("Accuracy vs Validation Accuracy")
plt.show()


### **Resnet50**

In [ ]:
from tensorflow.keras.models import Model,load_model, Sequential
from tensorflow.keras.layers import  GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import  Adam

from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
base_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (360, 640, 3))
resnet = base_model.output
resnet = GlobalAveragePooling2D()(resnet)
resnet = Dropout(0.25)(resnet)
predictions = Dense(3, activation= 'softmax')(resnet)
Resnet50 = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
adam = Adam(learning_rate = 0.001)
Resnet50.compile(optimizer= adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
History = Resnet50.fit(train_images,
                        train_labels, 
                        batch_size = 4,
                        epochs = 2
                        )

In [ ]:
test_loss2 = Resnet50.evaluate(test_images, test_labels)

# **Menghitung Visual Attributes Thumbnail**

## **Brightness**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import cv2
import os
import PIL.Image
import imquality.brisque as brisque

In [ ]:
from PIL import Image
import math
import glob

image_list = []
brightnes_list = []
# Open the image file
# for filename in glob.glob('E:/00 KULYAH/SEMESTER 5/Data Mining II/UAS/Projek UAS/all/*.jpg'):
for filename in path:
  im = Image.open(filename)
  image_list.append(im)
# Calculate the brightness using the formula:
# brightness = sqrt(0.241 * R^2 + 0.691 * G^2 + 0.068 * B^2)
# where R, G, and B are the red, green, and blue channels of the image
  width, height = im.size
  r_total = 0
  g_total = 0
  b_total = 0
  for x in range(width):
      for y in range(height):
          r, g, b = im.getpixel((x, y))
          r_total += r**2
          g_total += g**2
          b_total += b**2
  brightness = math.sqrt(0.241 * r_total + 0.691 * g_total + 0.068 * b_total)
  brightnes_list.append(brightness)

In [ ]:
data_complete = data_complete.assign(brightness=brightnes_list)
data_complete

## **Complexity**

In [ ]:
import cv2
from sklearn.ensemble import RandomForestClassifier
image_list = []
complexity_list = []
# Open the image file
for filename in path :
  im = Image.open(filename)
  image = cv2.imread(filename)
  image_list.append(im)
# Load gambar dan konversi ke skala abu-abu
# image = cv2.imread('WhatsApp Image 2022-12-03 at 00.32.45.jpg')
  gambar_abu_abu = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Gunakan algoritma deteksi tepi Canny untuk menemukan tepi dalam gambar
  tepi = cv2.Canny(gambar_abu_abu, 100, 200)
  tepi = tepi.reshape(-1,1)

# Gunakan Random Forest classifier untuk mengklasifikasi tepi sebagai sederhana atau kompleks
  classifier = RandomForestClassifier()
  classifier.fit(tepi, [0 if edge > 0 else 1 for edge in tepi])

# Hitung skor kompleksitas komponen dengan mencari rata-rata prediksi classifier
  skor_kompleksitas = classifier.predict(tepi).mean()
  complexity_list.append(skor_kompleksitas)

In [ ]:
data_complete = data_complete.assign(complexity=complexity_list)
data_complete

## **Colorfulness**

In [ ]:
image_list = []
colorfulness_list = []
# Open the image file
for filename in path:
  im = Image.open(filename)
  image = cv2.imread(filename)
  image_list.append(im)

# def image_colorfulness(image):
	# split the image into its respective RGB components
  (B, G, R) = cv2.split(image.astype("float"))
	# compute rg = R - G
  rg = np.absolute(R - G)
    # compute yb = 0.5 * (R + G) - B
  yb = np.absolute(0.5 * (R + G) - B)
  # compute the mean and standard deviation of both `rg` and `yb`
  (rbMean, rbStd) = (np.mean(rg), np.std(rg))
  (ybMean, ybStd) = (np.mean(yb), np.std(yb))
  # combine the mean and standard deviations
  stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
  meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
  # derive the "colorfulness" metric and return it
  score = stdRoot + (0.3 * meanRoot)
  colorfulness_list.append(score)

In [ ]:
data_complete = data_complete.assign(colorfulness=colorfulness_list)
data_complete

## **Quality**

In [ ]:
IMAGE_SIZE = (640, 360)

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, IMAGE_SIZE) 
        if img is not None:
            images.append(img)
            
    return np.array(images, dtype='uint8')

In [ ]:
def brisque_score(images):
    score = []
    for i in images:
        score.append(brisque.score(i))
    return score

In [ ]:
r = load_images_from_folder('train/Rendah/')
s = load_images_from_folder('train/Sedang/')
t = load_images_from_folder('train/Tinggi/')

In [ ]:
brisque_rendah = brisque_score(r)
brisque_sedang = brisque_score(s)
brisque_tinggi = brisque_score(t)
re = os.listdir('train/Rendah/')
se = os.listdir('train/Sedang/')
ti = os.listdir('train/Tinggi/')

In [ ]:
df_r = pd.DataFrame({'Filename': re,
                    'Label' : ['Rendah']*len(re),
                    'BRISQUE': brisque_rendah})
df_r.head()

In [ ]:
df_s = pd.DataFrame({'Filename': se,
                    'Label' : ['Sedang']*len(se),
                    'BRISQUE': brisque_sedang})
df_s.head()

In [ ]:
df_t = pd.DataFrame({'Filename': ti,
                    'Label' : ['Tinggi']*len(ti),
                    'BRISQUE': brisque_tinggi})
df_t.head()

# **Analisis Regresi**

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import stats
from statsmodels.compat import lzip
import statsmodels
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv("data_regresi.csv")
df.head()

In [ ]:
df = df.drop(['Unnamed: 0', 'name','path'], axis=1)
df

In [ ]:
import matplotlib.pyplot as plt

sns.lineplot(data=df, x="brightness", y="Detail Views", err_style="bars",ci=50)

In [ ]:
sns.lineplot(data=df, x="complexity", y="Detail Views", err_style="bars",ci=50)

In [ ]:
sns.lineplot(data=df, x="colorfulness", y="Detail Views", err_style="bars",ci=50)

In [ ]:
sns.lineplot(data=df, x="quality", y="Detail Views", err_style="bars",ci=50)

In [ ]:
df.isna().sum()

In [ ]:
# ====== Analisis Deskriptif ======
df.describe()

In [ ]:
desc.round(2)

**Visualisasi**

In [ ]:
#untuk mengetahui korelasi linier antar variabe
mask = np.triu(np.ones_like(df.corr())) #agar yang muncul segitiga bawah
annot = True #untuk mencetak/ menampilkan nilai korelasi di dalam kotak
sns.set(rc={'figure.figsize':(13,6)}) 
sns.heatmap(df.corr(method='pearson').round(2), annot=True, cmap='OrRd', mask=mask)

In [ ]:
plt.figure(figsize=(16,10)) 
sns.set_palette('Pastel1')
plt.subplot(2,2,1)                                                                                                #bikin 2 baris, 3 kolom, kasi di indeks 1
sns.regplot(x='brightness', y='Detail Views', data=df,line_kws={'color':'black'})
plt.subplot(2,2,2)
sns.regplot(x='complexity', y='Detail Views', data=df,line_kws={'color':'black'})
plt.subplot(2,2,3)
sns.regplot(x= 'colorfulness', y='Detail Views', data=df,line_kws={'color':'black'})
plt.subplot(2,2,4)
sns.regplot(x='quality', y='Detail Views', data=df,line_kws={'color':'black'})
plt.show()

In [ ]:
plt.figure(figsize=(13,13)) 
sns.set_palette('Spectral_r')
plt.subplot(3,2,1)                                                                                                #bikin 2 baris, 3 kolom, kasi di indeks 1
sns.distplot(df[ 'Detail Views'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,2)
sns.distplot(df['brightness'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,3)
sns.distplot(df['colorfulness'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,4)
sns.distplot(df['complexity'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,5)
sns.distplot(df[ 'quality'], bins=30, kde_kws={'linewidth': 5})
# plt.subplot(3,2,6)
# sns.distplot(df['price_share_usd'], bins=30, kde_kws={'linewidth': 5})
plt.show()

**Preprocessing**

In [ ]:
def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column]  / df_scaled[column].abs().max()
    return df_scaled
    
# call the maximum_absolute_scaling function
df_cars_scaled = maximum_absolute_scaling(df_cars)

df_cars_scaled

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

# create an abs_scaler object
abs_scaler = MaxAbsScaler()

# calculate the maximum absolute value for scaling the data using the fit method
abs_scaler.fit(df_cars)

# the maximum absolute values calculated by the fit method
abs_scaler.max_abs_
# array([4.0e+05, 1.7e+01])

# transform the data using the parameters calculated by the fit method (the maximum absolute values)
scaled_data = abs_scaler.transform(df_cars)

# store the results in a data frame
df_scaled = pd.DataFrame(scaled_data, columns=df_cars.columns)

# visualize the data frame
df_scaled

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm = pd.DataFrame(scaler.fit_transform(df_cars), columns=df_cars.columns)

df_norm

In [ ]:
# apply the z-score method in Pandas using the .mean() and .std() methods
def z_score(df):
    # copy the dataframe
    df_std = df.copy()
    # apply the z-score method
    for column in df_std.columns:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
        
    return df_std
    
# call the z_score function
df_cars_standardized = z_score(df_cars)

df_cars_standardized

In [ ]:
from sklearn.preprocessing import RobustScaler
# create a scaler object
scaler = RobustScaler()
# fit and transform the data
df_x = df[['brightness', 'complexity', 'colorfulness','quality']]
df_robust = pd.DataFrame(scaler.fit_transform(df_x), columns=df_x.columns)
df_robust

In [ ]:
plt.figure(figsize=(13,13)) 
sns.set_palette('Spectral_r')
plt.subplot(3,2,1)                                                                                                #bikin 2 baris, 3 kolom, kasi di indeks 1
sns.distplot(df[ 'Detail Views'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,2)
sns.distplot(df_robust['brightness'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,3)
sns.distplot(df_robust['colorfulness'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,4)
sns.distplot(df_robust['complexity'], bins=30, kde_kws={'linewidth': 5})
plt.subplot(3,2,5)
sns.distplot(df_robust[ 'quality'], bins=30, kde_kws={'linewidth': 5})
# plt.subplot(3,2,6)
# sns.distplot(df['price_share_usd'], bins=30, kde_kws={'linewidth': 5})
plt.show()

In [ ]:
scaler = RobustScaler()
# fit and transform the data
df_x = df[['brightness', 'complexity', 'colorfulness','quality']]
df[['brightness', 'complexity', 'colorfulness','quality']] = scaler.fit_transform(df_x)
df

In [ ]:
df.skew()

In [ ]:
t_sqrt = np.sqrt(df['Detail Views'])
t_sqrt.hist()

In [ ]:
t_ln = np.log(df['Detail Views'])
t_ln.hist()

In [ ]:
t_ln.skew()

In [ ]:
import math
log10 = np.log10(df['Detail Views'])
log10.hist()

In [ ]:
import math
t_1 = 1/(df['Detail Views'])
t_1.hist()

In [ ]:
df['Detail Views']=np.log(df['Detail Views'])
df['Detail Views'].hist()

In [ ]:
sns.lineplot(data=df, x="complexity", y="Detail Views", err_style="bars",ci=50)

In [ ]:
# Analisis korelasi antar variabel dalam dataset
mask = np.triu(np.ones_like(df.corr())) #agar yang muncul segitiga bawah
annot = True #untuk mencetak/ menampilkan nilai korelasi di dalam kotak
sns.set(rc={'figure.figsize':(30,13)}) 
sns.heatmap(df.corr(method='pearson').round(2), annot=True, cmap='viridis', mask=mask)

## **OLS**

In [ ]:
import statsmodels.api as sm
from scipy import stats
#summary
x = df[['brightness','complexity','colorfulness','quality']]
y = df['Detail Views']

#summary
X = sm.add_constant(x)
result1 = sm.OLS(y, X).fit()
print(result1.summary())

In [ ]:
y_pred = result1.predict(X)

In [ ]:
MSE = np.square(np.subtract(y,y_pred)).mean()
MSE

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

In [ ]:
#Mengambil nilai residual dalam summary
residual = result1.resid
#Membuat scatter plot residual vs time order
#Membuat list dari time order dataset
time1=list(range(1,334+1))
print (time1)
#Menampilkan sebaran data dalam scatter plot residual vs time order
plt.scatter(x=time1, y=residual)
plt.xlabel('time order', fontsize=12)
plt.ylabel('residuals', fontsize=12)
plt.show()

In [ ]:
# membuat plot residuals vs fitted value
plt.scatter(x=result1.fittedvalues, y=residual)
plt.xlabel('fitted value', fontsize=12)
plt.ylabel('residuals', fontsize=12)
plt.show()

In [ ]:
# Uji Kolmogorov Smirnov untuk uji normalitas data residual
from scipy.stats import kstest
ks1=kstest(residual,'norm')
print('Statistic KS:', ks1.statistic.round(4))
print('P-value:', ks1.pvalue.round(4))
alpha = 0.05
if ks1.pvalue > alpha:
	print('Data Berdistribusi Normal (Gagal Tolak H0)')
else:
	print('Data Tidak Berdistribusi Normal (Tolak H0)')

## **Random Forest Regressor**

In [ ]:
#summary
X = df[['brightness','complexity','colorfulness','quality']]
y = df['Detail Views']

# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

# Train the model
regr = RandomForestRegressor(n_estimators = 1100, max_depth = 70, random_state = 101)
regr.fit(X_train, y_train.values.ravel())

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Make prediction
predictions = regr.predict(X_test)

result = X_test
result['Detail Views'] = y_test
result['Prediction'] = predictions.tolist()
result.head()

In [ ]:
# Import library for visualization
import matplotlib.pyplot as plt
# Define x axis
x_axis = X_test.brightness
# Build scatterplot
plt.scatter(x_axis, y_test, c = 'b', alpha = 1, marker = '.', label = 'Real')
plt.scatter(x_axis, predictions, c = 'r', alpha = 1, marker = '.', label = 'Predicted')
plt.xlabel('brightness')
plt.ylabel('Price')
plt.grid(color = '#D3D3D3', linestyle = 'solid')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

# Mean squared error (MSE)
mse = mean_squared_error(y_test.values.ravel(), predictions)

# R-squared scores
r2 = r2_score(y_test.values.ravel(), predictions)

# Print metrics
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Find the best parameters for the model
parameters = {
    'max_depth': [70, 80, 90, 100],
    'n_estimators': [900, 1000, 1100]
}
gridforest = GridSearchCV(regr, parameters, cv = 3, n_jobs = -1, verbose = 1)
gridforest.fit(X_train, y_train)
gridforest.best_params_

In [ ]:
# Get features list
characteristics = X.columns
# Get the variables importances, sort them, and print the result
importances = list(regr.feature_importances_)
characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances];

In [ ]:
# Visualize the variables importances
plt.bar(characteristics, importances, orientation = 'vertical')
plt.xticks(rotation = 'vertical')
plt.ylabel('Importance')
plt.xlabel('Variable')
plt.grid(axis = 'y', color = '#D3D3D3', linestyle = 'solid')
plt.show()

sc : https://www.datacareer.de/blog/random-forest-in-python-with-scikit-learn/

## **SVM Regressor**

In [ ]:
import numpy as np
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(21)
N = 1000    
def makeData(x):    
    r = [a/10 for a in x]
    y = np.sin(x)+np.random.uniform(-.5, .2, len(x))
    return np.array(y+r)

x = [i/100 for i in range(N)]
y = makeData(x)
x = np.array(x).reshape(-1,1)

plt.scatter(x, y, s=5, color="blue")
plt.show()

In [ ]:
#summary
X = df[['brightness','complexity','colorfulness','quality']]
y = df['Detail Views']

svr = SVR().fit(X, y)
print(svr)

yfit = svr.predict(X)
score = svr.score(X,y)
print("R-squared:", score)
print("MSE:", mean_squared_error(y, yfit))

## **ANN Regressor**

In [ ]:
# Separate Target Variable and Predictor Variables
TargetVariable=['Detail Views']
Predictors=['brightness','complexity','colorfulness','quality']

X=df[Predictors].values
y=df[TargetVariable].values
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)
# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)
# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Installing required libraries
!pip install tensorflow
!pip install keras

In [1]:
from keras.models import Sequential
from keras.layers import Dense
# create ANN model
model = Sequential()
# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=4, kernel_initializer='normal', activation='relu'))
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))
# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

ModuleNotFoundError: No module named 'keras'

In [ ]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)

            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

In [ ]:
%matplotlib inline
ResultsData.plot(x='Parameters', y='Accuracy', figsize=(15,4), kind='line')

In [ ]:
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 15, epochs = 5, verbose=0)

# Generating Predictions on testing data
Predictions=model.predict(X_test)

# Scaling the predicted Price data back to original price scale
Predictions=TargetVarScalerFit.inverse_transform(Predictions)

# Scaling the y_test Price data back to original price scale
y_test_orig=TargetVarScalerFit.inverse_transform(y_test)

# Scaling the test data back to original scale
Test_Data=PredictorScalerFit.inverse_transform(X_test)

TestingData=pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Detail Views']=y_test_orig
TestingData['Predicted Detail Views']=Predictions
TestingData.head()

In [ ]:
# Computing the absolute percent error
APE=100*(abs(TestingData['Detail Views']-TestingData['Predicted Detail Views'])/TestingData['Detail Views'])
TestingData['APE']=APE

print('The Accuracy of ANN model is:', 100-np.mean(APE))
TestingData.head()

In [ ]:
# Function to generate Deep ANN model 
def make_regression_ann(Optimizer_trial):
    from keras.models import Sequential
    from keras.layers import Dense
    
    model = Sequential()
    model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

###########################################
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

# Listing all the parameters to try
Parameter_Trials={'batch_size':[10,20],
                      'epochs':[10,20],
                    'Optimizer_trial':['adam', 'rmsprop']
                 }

# Creating the regression ANN model
RegModel=KerasRegressor(make_regression_ann, verbose=0)

###########################################
from sklearn.metrics import make_scorer

# Defining a custom function to calculate accuracy
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

#########################################
# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

#########################################
# Measuring how much time it took to find the best params
import time
StartTime=time.time()

# Running Grid Search for different paramenters
grid_search.fit(X,y, verbose=0)

EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')

print('### Printing Best parameters ###')
grid_search.best_params_

In [ ]:
from sklearn.metrics import mean_squared_error
  
# Given values
Y_true = TestingData['Detail Views']
  
# calculated values
Y_pred = TestingData['Predicted Detail Views']
  
# Calculation of Mean Squared Error (MSE)
print('Nilai MSE: ',mean_squared_error(Y_true,Y_pred))